In [1]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import pandas as pd


/root/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[4pdvGPU Msg(72319:139897580648256:libvgpu.c:869)]: Initializing.....
[4pdvGPU Warn(72319:139897580648256:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72319:139897580648256:hook.c:475)]: remap handles for device 1


In [2]:
csv_list = ["mcq_sing_cpa.csv","mcq_sing_lbk.csv","mcq_sing_nje.csv","mcq_sing_pae.csv",
           "mcq_sing_pfe.csv","mcq_sing_ungee.csv"]

### base model

In [3]:
model_name = "../model/Qwen2.5-3B-Instruct"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Warn(72319:139897580648256:utils.c:228)]: get default cuda 2 from (null)
[4pdvGPU Msg(72319:139897580648256:libvgpu.c:902)]: Initialized
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards:   0%|                     | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=24971051008 total=25217466368 limit=25769803776 usage=236978176
Loading checkpoint shards: 100%|█████████████| 2/2 [00:03<00:00,  1.66s/it]
[4pdvGPU Warn(72404:140421367076672:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72404:140421367076672:hook.c:475)]: remap handles for device 1


In [4]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content[0] ==  df["output"][i]:
            acc += 1    
        answer.append(content[0])
    print(acc)
persentage = acc/total_len
print(persentage)

134
340
583
638
766
991
0.6128633271490415


In [5]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

True

### cot_model

In [6]:
model_name = "../model/law_3b_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21152137216 total=25217466368 limit=25769803776 usage=4003595264
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21745631232 total=25217466368 limit=25769803776 usage=3410101248
Loading checkpoint shards:   0%|                     | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21152137216 total=25217466368 limit=25769803776 usage=4003595264
[4pdvGPU Msg(72319:139897580648256:memory.c:566)]: orig free=21745631232 total=25217466368 limit=25769803776 usage=3410101248
Loading checkpoint shards: 100%|█████████████| 2/2 [00:04<00:00,  2.42s/it]
[4pdvGPU Warn(72500:140335937480512:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72500:140335937480512:hook.c:475)]: remap handles for device 1


In [ ]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content[0] ==  df["output"][i]:
            acc += 1    
        answer.append(content[0])
    print(acc)
persentage = acc/total_len
print(persentage)

134
336


In [ ]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

### full_model

In [8]:
model_name = "../model/law_3b_full_data_f16"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

[4pdvGPU Msg(71989:139862630549312:memory.c:566)]: orig free=17440178176 total=25217466368 limit=25769803776 usage=7715554304
[4pdvGPU Msg(71989:139862630549312:memory.c:566)]: orig free=19254214656 total=25217466368 limit=25769803776 usage=5901517824
Loading checkpoint shards:   0%|                                        | 0/2 [00:00<?, ?it/s][4pdvGPU Msg(71989:139862630549312:memory.c:566)]: orig free=17440178176 total=25217466368 limit=25769803776 usage=7715554304
[4pdvGPU Msg(71989:139862630549312:memory.c:566)]: orig free=19254214656 total=25217466368 limit=25769803776 usage=5901517824
Loading checkpoint shards: 100%|████████████████████████████████| 2/2 [00:04<00:00,  2.31s/it]
[4pdvGPU Warn(72213:140549654329152:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(72213:140549654329152:hook.c:475)]: remap handles for device 1


In [9]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    total_len += len(df)
    for i in range(len(df)):
        prompt = f"""
        这是一个关于法律问题的单项选择题,请根据题目选择正确的选项(只需要回答正确选项即可，不需要进行分析)：\n
        问题：{df["input"][i]}\n
        A：{df["A"][i]}\n
        B:{df["B"][i]}\n
        C:{df["C"][i]}\n
        D:{df["D"][i]}\n
        """
        messages = [
            {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
        
        # conduct text completion
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=50
        )
        
        content = tokenizer.decode(generated_ids[0][model_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip("\n")
        if content[0] ==  df["output"][i]:
            acc += 1    
        answer.append(content[0])
    print(acc)
persentage = acc/total_len
print(persentage)

145
363
607
661
799
1031
0.6376004947433519


In [18]:
path = "../data/val_csv_single/"
total_len=0
answer = []
acc = 0
for name in csv_list:
    csv_name = "".join([path,name])
    df = pd.read_csv(csv_name)
    print(len(df))


197
275
537
118
170
320


In [13]:
def contains_only_allowed_chars(lst, allowed_chars={'A', 'B', 'C', 'D'}):
    # 将列表转换为大写，并检查是否是允许字符的子集
    return set(x.upper() for x in lst).issubset(allowed_chars)
contains_only_allowed_chars(answer)

True